# **Educación y Nivel Socioeconómico**
## Proyecto del módulo de Procesamiento de Datos con Python
### Data Analysis (Santander) - BEDU
## Equipo 19
- Orlando Aguilar
- Mario Encinas
- Andrés Sánchez

# Introducción

Según datos del Banco Mundial, México es un país con un nivel socioeconómico medio-alto, y es una de las principales economías de Latinoamérica. Sin embargo, el pormcentaje de población que vive en pobreza es alarmantemente alto.

De acuerdo con información proporcionada por el CONEVAL en 2018, el porcentaje de los mexicanos viviendo con ingresos inferiores a la línea de pobreza corresponde al 48.8% de la población.
Es necesario aclarar que el CONEVAL mide la línea de pobreza por ingresos a partir de los ingresos mensuales totales por hogar y el número de habitantes en el hogar.

La línea de pobreza utilizada por el CONEVAL en 2018 se localizaba alrededor de los 3,300 MXN mensuales para las zonas urbanas y alrededor de 2,300 MXN mensuales para las zonas rurales. De modo que un hogar con tres habitantes y un ingreso total de 15,000 MXN mensuales estaría por encima de la línea de pobreza, mientras que un hogar con cinco habitantes y un ingreso total de 15,000 MXN mensuales se encontraría en el umbral de pobreza en México. 

Un estudio realizado en 2009 por la CEPAL determinó que existe una correlación entre la población en situación de pobreza y un menor nivel de estudios. Se detectó que un individuo con bajo nivel educativo tiene más probabilidades de encontrarse en situación de pobreza. No obstante, el mismo estudio declara que no fue posible determinar si un mayor nivel de estudios permite a las personas salir de la pobreza.

Tomando en cuenta la información presentada previamente se decidió realizar un análisis para determinar si existe una correlación directa entre el nivel educativo y el nivel de ingresos de un individuo, y de ser así, cómo es esta relación.

Con el análisis propuesto se espera obtener información que proporcione una mejor comprensión del problema de la pobreza y la educación en México, pues es necesario conocerlo a fondo para poder desarrollar estrategias eficaces que permitan impulsar el desarrollo del país.

### Preguntas de interés:

- ¿existe una correlación entre el nivel educativo y el nivel de ingresos?
- ¿La educación universitaria garantiza un mayor nivel socioeconómico?
- ¿En qué medida afecta el nivel socioeconómico a las posibilidades de estudiar?
- ¿Cuál es la relación entre el nivel socioeconómico de los individuos con un cierto grado de estudios respecto a otro?
- ¿Hay una diferencia en los niveles de ingresos de los individuos con únicamente estudios de nivel primaria respecto a los de secundaría?
- ¿El nivel de estudios de un país aumenta en la misma medida que disminuye la pobreza?


# Desarrollo

In [ ]:
!pip install world_bank_data --upgrade

In [49]:
import csv
import pandas as pd
import numpy as np
import world_bank_data as wb

## **Recolección de Datos**


Los datos utilizados en este trabajo fueron extraídos de 2 fuentes principales: El Banco Mundial, y los resultados de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) de 2018 del INEGI. Los archivos descargados se encuentran en el [repositorio del proyecto](https://github.com/andresbsa/BEDU_Python/blob/main/README.md).

Se cargan 3 datasets diferentes de la ENIGH:


In [ ]:
concentrado_hogar = pd.read_csv('https://raw.githubusercontent.com/andresbsa/BEDU_Python/main/conjunto_de_datos_concentradohogar_enigh_2018_ns.csv')
datos_poblacion = pd.read_csv('https://raw.githubusercontent.com/andresbsa/BEDU_Python/main/conjunto_de_datos_poblacion_enigh_2018_ns.csv')
datos_ingresos = pd.read_csv('https://raw.githubusercontent.com/andresbsa/BEDU_Python/main/conjunto_de_datos_ingresos_enigh_2018_ns.csv')

- El DataFrame `concentrado_hogar` incluye información general de las viviendas mexicanas. Las columnas de interés son las relacionadas con el nivel socioeconómico, ingresos que recibidos y datos concentrados respecto a los habitantes.

- El DataFrame `datos_población` incluye información sociodemográfica de las viviendas a nivel integrante.

- El DataFrame `datos_ingresos` incluye los ingresos que recibió cada integrante de vivienda a lo largo de 6 meses.



Se cargan los datos del Banco Mundial (la api no requiere token): 

In [52]:
#wb.get_countries(country='MEX')
pobreza_nacional = wb.get_series('SI.POV.NAHC', id_or_value='id', country='mx')
pobreza_monetaria_moderada = wb.get_series('SI.POV.UMIC', id_or_value='id', country='MEX')#<5.50 USD por día
pobreza_monetaria_extrema = wb.get_series('SI.POV.DDAY', id_or_value='id', country='MEX') #<1.90 USD por día
secundaria_respecto_primaria = wb.get_series('SE.SEC.PROG.ZS', id_or_value='id', country='MEX')
primaria_respecto_rango_edad = wb.get_series('SE.PRM.ENRR', id_or_value='id', country='MEX')
secundaria_respecto_rango_edad = wb.get_series('SE.SEC.ENRR', id_or_value='id', country='MEX')
terciaria_respecto_rango_edad = wb.get_series('SE.TER.ENRR', id_or_value='id', country='MEX')

- La Serie `pobreza_nacional` contiene los porcentajes de población en situación pobreza a lo largo de los años.
- La Serie `pobreza_monetaria_moderada` contiene los porcentajes de población en situación de pobreza monetaria moderada a lo largo de los años; es decir, el porcentaje de personas en México que subsisten con menos de 5.50 USD al día.
- La Serie `pobreza_monetaria_extrema` contiene los porcentajes de población en situación de pobreza monetaria extrema a lo largo de los años; es decir, el porcentaje de personas en México que subsisten con menos de 1.90 USD al día.
- La Serie `secundaria_respecto_primaria` porcentaje de individuos que ingresan al nivel educativo secundario a lo largo de los años. Se obtiene a partir del numero de individuos que egresan del nivel primario respecto al numero que ingresa al nivel secundario en el año indicado. 
- La Serie `primaria_respecto_rango_edad` porcentaje de individuos que ingresan al nivel educativo primario a lo largo de los años. Se obtiene a partir del número de individuos que ingresan respecto al número de personas a nivel nacional en el rango de edad estimado para ingresar a dicho grado educativo.
- La Serie `secundaria_respecto_rango_edad` porcentaje de individuos que ingresan al nivel educativo secundario a lo largo de los años. Se obtiene a partir del número de individuos que ingresan respecto al número de personas a nivel nacional en el rango de edad estimado para ingresar a dicho grado educativo.
- La Serie `terciaria_respecto_rango_edad` porcentaje de individuos que ingresan al nivel educativo terciario a lo largo de los años. Se obtiene a partir del número de individuos que ingresan respecto al número de personas a nivel nacional en el rango de edad estimado para ingresar a dicho grado educativo.


## **Limpieza de Datos**

### Indicadores del Banco Mundial


Los indicadores descargados del Banco Mundial se almacenan en series de tiempo correspondientes a distintos parámetros hacerca de Educación y Pobreza. Estos datos se convierten en Series de `pandas` con sus respectivos nombres y reestablecen sus índices sin soltar las columnas correspondientes. Posteriormente, se genera un solo DataFrame llamado `indicadores`.

Con el DataFrame generado, se destina la columna que contiene los años como el índice. Dado que el rango de tiempo de todas las series es exactamente el mismo, se puede tomar la columna `Year` de cualquiera de las series.


In [53]:
anios = pd.Series(pobreza_nacional.reset_index()['Year'])
pobreza_nacional = pd.Series(pobreza_nacional.reset_index()['SI.POV.NAHC'])
pobreza_moderada = pd.Series(pobreza_monetaria_moderada.reset_index()['SI.POV.UMIC'])
pobreza_extrema = pd.Series(pobreza_monetaria_extrema.reset_index()['SI.POV.DDAY'])
secundaria_respecto_primaria = pd.Series(secundaria_respecto_primaria.reset_index()['SE.SEC.PROG.ZS'])
primaria_respecto_rango_edad = pd.Series(primaria_respecto_rango_edad.reset_index()['SE.PRM.ENRR'])
secundaria_respecto_rango_edad = pd.Series(secundaria_respecto_rango_edad.reset_index()['SE.SEC.ENRR'])
terciaria_respecto_rango_edad = pd.Series(terciaria_respecto_rango_edad.reset_index()['SE.TER.ENRR'])

indicadores = pd.DataFrame({'Año': anios, 'Pobreza_nacional': pobreza_nacional, 'Pobreza_moderada': pobreza_moderada,
                                     'Pobreza_extrema': pobreza_extrema, 'Primaria_respecto_rango_edad': primaria_respecto_rango_edad,
                                     'Secundaria_respecto_rango_edad': secundaria_respecto_rango_edad,
                                     'Secundaria_respecto_primaria': secundaria_respecto_primaria,
                                     'terciaria_respecto_rango_edad': terciaria_respecto_rango_edad})

indicadores = indicadores.set_index('Año', drop=True)
indicadores

,Pobreza_nacional,Pobreza_moderada,Pobreza_extrema,Primaria_respecto_rango_edad,Secundaria_respecto_rango_edad,Secundaria_respecto_primaria,terciaria_respecto_rango_edad
Año,,,,,,,
1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2016,43.6,25.4,2.2,106.277733,102.404060,96.57231,38.433380
2017,NaN,NaN,NaN,105.770012,104.385033,NaN,40.228958
2018,41.9,22.7,1.7,105.028069,105.103363,NaN,41.522800


Una vez con el DataFrame generado, se eliminan todos los registros cuyos valores corresponden exclusivamente a NaNs.

Posteriormente comprobamos cuantos registros NaN contiene cada columna.

In [54]:
indicadores = indicadores.dropna(axis=0, how='all')
indicadores = indicadores.dropna(axis = 0, subset=['terciaria_respecto_rango_edad'])
indicadores.isna().sum(axis=0)

Pobreza_nacional                  40
Pobreza_moderada                  30
Pobreza_extrema                   30
Primaria_respecto_rango_edad       0
Secundaria_respecto_rango_edad     0
Secundaria_respecto_primaria      10
terciaria_respecto_rango_edad      0
dtype: int64

Los valores nulos en las columnas `Pobreza_moderada` y `Pobreza_extrema` coinciden en registros, por lo que se procede a crear un DataFrame eliminando los registros con valores NaN en las columnas mencionadas.
Debido a la escasa cantidad de registros restantes se eliminan las columnas `Pobreza_nacional` y `Secundaria_respecto_primaria` pues estas siguen conteniendo valores NaN.


In [55]:
indicadores_pobreza_y_educacion = indicadores.dropna(axis = 0, subset=['Pobreza_moderada', 'Pobreza_extrema'], how='all')
indicadores_pobreza_y_educacion = indicadores_pobreza_y_educacion.drop(columns=['Secundaria_respecto_primaria', 'Pobreza_nacional'])
print(indicadores_pobreza_y_educacion.isna().sum(axis=0))
print(len(indicadores_pobreza_y_educacion))

Pobreza_moderada                  0
Pobreza_extrema                   0
Primaria_respecto_rango_edad      0
Secundaria_respecto_rango_edad    0
terciaria_respecto_rango_edad     0
dtype: int64
15


Para no perder tanta información acerca de los indicadores de educación se crea un segundo DataFrame que almacena únicamente las columnas correspondientes a los indicadores de educación y se elimina cualquier registro con un valor NaN.

In [56]:
indicadores_educacion = indicadores.dropna(axis = 0, subset=['Secundaria_respecto_primaria'], how='any')
indicadores_educacion = indicadores_educacion.drop(columns=['Pobreza_nacional', 'Pobreza_moderada', 'Pobreza_extrema'])
print(indicadores_educacion.isna().sum(axis=0))
print(len(indicadores_educacion))

Primaria_respecto_rango_edad      0
Secundaria_respecto_rango_edad    0
Secundaria_respecto_primaria      0
terciaria_respecto_rango_edad     0
dtype: int64
35


Se espera que con los DataFrames obtenidos a partir de los datos del Banco Mundial se obtenga información que permita tener una mejor comprensión del efecto de la pobreza económica sobre el acceso a la educación.

### Concentrado Hogar
Del DataFrame `concentrado_hogar` sobre las viviendas mexicanas y su nivel socioeconómico, aislamos las columnas relevantes para los objetivos de la investigación. 

Primero guardamos todos los nombres de las columnas en `columnas_concentrado` y definimos una lista con los índices de las columnas de interés en `numero_columnas`. Con este mapeo, se filtran las columnas del DataFrame original en uno nuevo llamado `concentrado`. Asimismo, se guardan nuevos nombres para las columnas del DataFrame en la lista `nombres_columnas` y se reemplazan en el DataFrame filtrado. 

Por último, se observan el tipo de datos en el DataFrame filtrado y se identifican cuantos NaNs contiene.

In [57]:
columnas_concentrado = concentrado_hogar.columns
numero_columnas = [0,1,2,3,4,5,6,7,8,12,15,16,17,18,19,20,22]
concentrado = concentrado_hogar[columnas_concentrado[numero_columnas]]

nombres_columnas = ['folio_vivienda', 'numero_hogar', 'ubicacion_geografica', 'tamanio_localidad', 'nivel_socioeconomico', 
                    'estrato_dis', 'upm', 'factor', 'clase_hogar', 'total_integrantes', 'mayores_11', 'menores_12',
                    'edad_11_a_64', 'mayores 65', 'ocupados', 'perciben_ingresos', 'ingresos_coordinados']
concentrado.columns = nombres_columnas

print(concentrado.dtypes)
print(concentrado.isna().sum(axis=0))

folio_vivienda            int64
numero_hogar              int64
ubicacion_geografica      int64
tamanio_localidad         int64
nivel_socioeconomico      int64
estrato_dis               int64
upm                       int64
factor                    int64
clase_hogar               int64
total_integrantes         int64
mayores_11                int64
menores_12                int64
edad_11_a_64              int64
mayores 65                int64
ocupados                  int64
perciben_ingresos         int64
ingresos_coordinados    float64
dtype: object
folio_vivienda          0
numero_hogar            0
ubicacion_geografica    0
tamanio_localidad       0
nivel_socioeconomico    0
estrato_dis             0
upm                     0
factor                  0
clase_hogar             0
total_integrantes       0
mayores_11              0
menores_12              0
edad_11_a_64            0
mayores 65              0
ocupados                0
perciben_ingresos       0
ingresos_coordinados    0


### Población
El mismo proceso de filtrado y renombramiento se sigue con el DataFrame `datos_poblacion` sobre la sociodemografía de las viviendas por cada integrante: se filtran las columnas revelantes con la lista `numero_columnas` y se renombran con `nombres_columnas`

Después se observan el tipo de datos del DataFrame filtrado y se identifican cuántos NaNs contiene.

In [58]:
columnas_poblacion = datos_poblacion.columns
numero_columnas = [0,2,3,4,5,10,28,29,30,31,33,34,36,37,39,40,42,47,48,49,50,52,56,58,59,61]
poblacion = datos_poblacion[columnas_poblacion[numero_columnas]]

nombres_columnas = ['folio_vivienda', 'numero_habitante', 'parentesco', 'sexo', 'edad', 'discapacidad',
                    'pertenece_etnia', 'alfabetismo', 'asiste_a_escuela', 'grado_estudios', 'tipo_escuela', 'tiene_beca', 'tipo_beca',
                    'tiene_ayuda', 'tipo_ayuda', 'grado_maximo_estudios', 'antecedentes_estudios', 'aporta_seguridad_social(ss)',
                    'anios_contribucion_ss', 'meses_contribucion_ss', 'apoyo_conseguir_trabajo', 'apoyo_prestamo_mes_gastos',
                    'horas_trabajo', 'no_recuerda_trabajar', 'horas_estudio', 'no_recuerda_estudiar']

poblacion.columns = nombres_columnas

print(poblacion.dtypes)
print(poblacion.isna().sum(axis=0))

folio_vivienda                  int64
numero_habitante                int64
parentesco                      int64
sexo                            int64
edad                            int64
discapacidad                   object
pertenece_etnia                object
alfabetismo                    object
asiste_a_escuela               object
grado_estudios                 object
tipo_escuela                   object
tiene_beca                     object
tipo_beca                      object
tiene_ayuda                    object
tipo_ayuda                     object
grado_maximo_estudios          object
antecedentes_estudios          object
aporta_seguridad_social(ss)    object
anios_contribucion_ss          object
meses_contribucion_ss          object
apoyo_conseguir_trabajo        object
apoyo_prestamo_mes_gastos      object
horas_trabajo                  object
no_recuerda_trabajar           object
horas_estudio                  object
no_recuerda_estudiar           object
dtype: objec

### Ingresos


Con el DataFrame `datos_ingresos` se siguió el mismo procedimiento que con los dos DataFrames anteriores para recuperar solo las columnas relevantes y renombrarlas.

El DataFrame filtrado se guarda como `ingresos`.

In [59]:
columnas_ingresos=datos_ingresos.columns
numero_columnas_ingresos = [0,1,2,3,10,11,12,13,14,15,16]
ingresos=datos_ingresos[columnas_ingresos[numero_columnas_ingresos]]
nombres_columnas_ingresos= ['folio_vivienda','numero_hogar','numero_habitante','clave_fuente',
                            'ingreso_1','ingreso_2','ingreso_3','ingreso_4','ingreso_5','ingreso_6', 'ingreso_trimestral']
ingresos.columns=nombres_columnas_ingresos

print(ingresos.dtypes)
print(ingresos.isna().sum(axis=0))


folio_vivienda          int64
numero_hogar            int64
numero_habitante        int64
clave_fuente           object
ingreso_1               int64
ingreso_2              object
ingreso_3              object
ingreso_4              object
ingreso_5              object
ingreso_6              object
ingreso_trimestral    float64
dtype: object
folio_vivienda        0
numero_hogar          0
numero_habitante      0
clave_fuente          0
ingreso_1             0
ingreso_2             0
ingreso_3             0
ingreso_4             0
ingreso_5             0
ingreso_6             0
ingreso_trimestral    0
dtype: int64


## **Transformación de Datos**

### Concentrado Hogar

El DataFrame `concentrado` no requiere de transformación de datos, pues todas las columnas ya se encuentran definidas en un tipo de datos apropiado y el DataFrame carece completamente de datos NaN.

### Población

Antes de realizar castings, imprimimos los sets de valores por columna, para conocer un poco sobre los posibles valores que toman nuestras columnas:

In [60]:
for columna in nombres_columnas[5:]:
  print(columna, set(poblacion[columna]))

discapacidad {1, '2', 2, 3, '5', 6, '&', '4', '3', 8, 4, 7, 5, '1', '6', '8', '7'}
pertenece_etnia {'2', '1', ' '}
alfabetismo {'2', '1', ' '}
asiste_a_escuela {'2', '1', ' '}
grado_estudios {'9', ' ', '4', '1', '12', '10', '13', '6', '11', '8', '7'}
tipo_escuela {'3', '2', '1', ' '}
tiene_beca {'2', '1', ' '}
tipo_beca {'2', '5', ' ', '3', '1', '4'}
tiene_ayuda {'2', '1', ' '}
tipo_ayuda {'3', '2', '1', ' '}
grado_maximo_estudios {'2', '5', '9', '3', ' ', '1', '6', '8', '4', '7', '0'}
antecedentes_estudios {'2', '5', '3', ' ', '1', '4'}
aporta_seguridad_social(ss) {'2', '1', ' '}
anios_contribucion_ss {'21', '5', '22', '19', '34', '3', '42', '25', '47', '61', '30', '38', '35', '11', '7', '49', '54', '48', '2', '36', '26', '9', ' ', '29', '10', '12', '44', '20', '28', '51', '55', '4', '45', '50', '16', '41', '23', '24', '6', '46', '14', '-1', '52', '62', '32', '31', '58', '40', '37', '17', '60', '53', '15', '1', '13', '43', '56', '27', '33', '39', '18', '65', '8', '0'}
meses_contribuci

En este DataFrame, la mayoría de los tipos de datos se pueden analizar con mayor facilidad en la forma de valores numéricos. Por ello, se modifica el tipo de dato con la función de pandas `to_numeric`. 

Con el parámetro 'errors' se sustituyen los valores que no se pueden convertir por NaNs. Debido a que los valores que no son números se reemplazan con NaNs, se vuelven a identificar cuántos de estos datos hay en el DataFrame.

In [61]:
for columna in nombres_columnas[5:]:
  poblacion[columna] = pd.to_numeric(poblacion[columna], errors='coerce')

print(poblacion.dtypes)
print(poblacion.isna().sum(axis=0))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


folio_vivienda                   int64
numero_habitante                 int64
parentesco                       int64
sexo                             int64
edad                             int64
discapacidad                   float64
pertenece_etnia                float64
alfabetismo                    float64
asiste_a_escuela               float64
grado_estudios                 float64
tipo_escuela                   float64
tiene_beca                     float64
tipo_beca                      float64
tiene_ayuda                    float64
tipo_ayuda                     float64
grado_maximo_estudios          float64
antecedentes_estudios          float64
aporta_seguridad_social(ss)    float64
anios_contribucion_ss          float64
meses_contribucion_ss          float64
apoyo_conseguir_trabajo        float64
apoyo_prestamo_mes_gastos      float64
horas_trabajo                  float64
no_recuerda_trabajar           float64
horas_estudio                  float64
no_recuerda_estudiar     

Posteriormente, con los datos en forma númerica, se procede a filtrarlos con base en algunas de las columnas. 

Primero, se filtran los registros de acuerdo a la edad de los individuos, descartando aquellas personas con 12 años o menos. Esta delimitación se hace para remover a las personas que en su mayoría no tienen porqué tener un ingreso. De esta forma, los datos se limitan al nivel educativo de aquellas personas que requieren de un ingreso.

Posteriormente se eliminan los registros que posean valores NaN en la columna `discapacidad` debido a que no es posible identificar el estado de salud de dichos individuos, y los registros correspondientes a individuos con un estado de salud incierto no permiten la extracción de información concluyente.

Para el parámetro `aporta_seguridad_social` se reemplazan los NaNs por el valor `2`, pues este numero indica en esta columna que el individuo no aporta a seguridad social. Es posible asignar el valor `2` a los registros NaN en esta columna pues se infiere que, si un individuo no aplica para aportar a seguridad social, no estará aportando. 

Para el resto del DataFrame, los registros con valores NaN se reemplazan con `0`, pues la naturaleza de dichas columnas permite la asignacion de el valor `0` sin perder significacia en los valores de los registros.

Una vez filtrado los registros y manejados los NaNs se reestablece el índice para que tenga continuidad y se vuelve a revisar si alguna columna contiene valores NaNs aún.

In [62]:
poblacion = poblacion.loc[poblacion['edad'] > 12]
#poblacion.isna().sum(axis=0)

#poblacion.loc[poblacion['discapacidad'].isna()]['edad'].describe()
poblacion = poblacion.dropna(axis = 0, subset=['discapacidad'])
#poblacion.isna().sum(axis=0)

#poblacion.loc[poblacion['aporta_seguridad_social(ss)'].isna()]['edad'].describe()
poblacion['aporta_seguridad_social(ss)'] = poblacion['aporta_seguridad_social(ss)'].fillna(2)
#poblacion.isna().sum(axis=0)

#poblacion.loc[poblacion['apoyo_conseguir_trabajo'].isna()]['edad'].describe()
#set(poblacion.loc[poblacion['apoyo_conseguir_trabajo'].isna()]['discapacidad'])
#set(poblacion.loc[poblacion['apoyo_conseguir_trabajo'].isna()]['apoyo_prestamo_mes_gastos'])
poblacion = poblacion.fillna(0)

poblacion = poblacion.reset_index(drop=True)
poblacion.isna().sum(axis=0)

folio_vivienda                 0
numero_habitante               0
parentesco                     0
sexo                           0
edad                           0
discapacidad                   0
pertenece_etnia                0
alfabetismo                    0
asiste_a_escuela               0
grado_estudios                 0
tipo_escuela                   0
tiene_beca                     0
tipo_beca                      0
tiene_ayuda                    0
tipo_ayuda                     0
grado_maximo_estudios          0
antecedentes_estudios          0
aporta_seguridad_social(ss)    0
anios_contribucion_ss          0
meses_contribucion_ss          0
apoyo_conseguir_trabajo        0
apoyo_prestamo_mes_gastos      0
horas_trabajo                  0
no_recuerda_trabajar           0
horas_estudio                  0
no_recuerda_estudiar           0
dtype: int64

En este punto, con los DataFrames `concentrado` y `poblacion` procesados se realiza un `merge` para manejar los datos relevantes en una misma tabla y facilitar el acceso a la información. 

El `merge` se hace sobre el DataFrame `poblacion`, introduciendo las variables sobre el tipo de localidad en el que la familia vive y su nivel socioeconómico. La unión se hace sobre el `folio_vivienda`, pues cada familia tiene su propio folio único. 

In [63]:
poblacion = poblacion.merge(concentrado[['tamanio_localidad', 'nivel_socioeconomico', 'folio_vivienda']], how='left', on='folio_vivienda')

### Ingresos

Convertimos a valor numérico las columnas correspondientes a ingresos:

In [ ]:
for columna in nombres_columnas_ingresos[5:]:
    ingresos[columna] = pd.to_numeric(ingresos[columna],errors='coerce')

In [65]:
print(ingresos.isna().sum(axis=0)) 

folio_vivienda            0
numero_hogar              0
numero_habitante          0
clave_fuente              0
ingreso_1                 0
ingreso_2             55094
ingreso_3             55094
ingreso_4             55094
ingreso_5             55094
ingreso_6             55094
ingreso_trimestral        0
dtype: int64


55094 registros de ingresos son valores NaNs. Dado que esta es una variable central, se reemplazan con 0´s.

In [66]:
ingresos_limpio = ingresos.fillna(0)
print(ingresos_limpio.isna().sum(axis=0)) 

folio_vivienda        0
numero_hogar          0
numero_habitante      0
clave_fuente          0
ingreso_1             0
ingreso_2             0
ingreso_3             0
ingreso_4             0
ingreso_5             0
ingreso_6             0
ingreso_trimestral    0
dtype: int64


Se tienen dos tablas relacionadas, `poblacion` e `ingresos_limpio`. Una contiene las características sociodemográficas de los integrantes de vivienda y la otra incluye los últimos 6 ingresos mensuales que percibieron.

Se utilizan las columnas `folio_vivienda` y `numero_habitante` para combinar ambos DataFrames con un `merge` en un DataFrame final para los datos recuperados del INEGI llamado `poblacion_completo`.


In [67]:
poblacion_laboral = poblacion.merge(ingresos_limpio, how='left', on=["folio_vivienda","numero_habitante"]).sort_index()

Se observa el DataFrame `poblacion_completo` para ver cuántos valores `null` hay.

In [68]:
poblacion_laboral.isnull().sum(axis=0) # 41270 de los integrantes de familia no tienen ingresos correspondientes

folio_vivienda                     0
numero_habitante                   0
parentesco                         0
sexo                               0
edad                               0
discapacidad                       0
pertenece_etnia                    0
alfabetismo                        0
asiste_a_escuela                   0
grado_estudios                     0
tipo_escuela                       0
tiene_beca                         0
tipo_beca                          0
tiene_ayuda                        0
tipo_ayuda                         0
grado_maximo_estudios              0
antecedentes_estudios              0
aporta_seguridad_social(ss)        0
anios_contribucion_ss              0
meses_contribucion_ss              0
apoyo_conseguir_trabajo            0
apoyo_prestamo_mes_gastos          0
horas_trabajo                      0
no_recuerda_trabajar               0
horas_estudio                      0
no_recuerda_estudiar               0
tamanio_localidad                  0
n

Se eliminan a los individuos que no cuentan con ingresos correspondientes del DataFrame final y se reasignan indices.


In [70]:
poblacion_laboral = poblacion_laboral.dropna(axis=0,how='any')
poblacion_laboral = poblacion_laboral.reset_index(drop=True)
poblacion_laboral

,folio_vivienda,numero_habitante,parentesco,sexo,edad,discapacidad,pertenece_etnia,alfabetismo,asiste_a_escuela,grado_estudios,tipo_escuela,tiene_beca,tipo_beca,tiene_ayuda,tipo_ayuda,grado_maximo_estudios,antecedentes_estudios,aporta_seguridad_social(ss),anios_contribucion_ss,meses_contribucion_ss,apoyo_conseguir_trabajo,apoyo_prestamo_mes_gastos,horas_trabajo,no_recuerda_trabajar,horas_estudio,no_recuerda_estudiar,tamanio_localidad,nivel_socioeconomico,numero_hogar,clave_fuente,ingreso_1,ingreso_2,ingreso_3,ingreso_4,ingreso_5,ingreso_6,ingreso_trimestral
0,100013601,1,101,1,74,8.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1.0,P032,3100.0,3100.0,3100.0,3100.0,3100.0,3100.0,9147.54
1,100013601,1,101,1,74,8.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1.0,P001,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,29508.19
2,100013601,2,201,2,70,8.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,9.0,0.0,9.0,1,3,1.0,P044,0.0,1100.0,0.0,1100.0,0.0,1100.0,1622.95
3,100013601,3,301,1,38,8.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,1.0,P001,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,23606.55
4,100013602,1,101,1,48,8.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,5.0,1.0,15.0,0.0,3.0,2.0,48.0,0.0,0.0,9.0,1,3,1.0,P070,4563.0,4163.0,4563.0,4163.0,4563.0,4163.0,12874.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361190,3260798905,1,101,1,59,8.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,3.0,3.0,45.0,0.0,0.0,9.0,4,2,1.0,P001,6800.0,8500.0,6800.0,6800.0,8500.0,6800.0,21619.56
361191,3260798906,1,101,1,88,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,9.0,0.0,9.0,4,2,1.0,P040,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,2934.78
361192,3260798906,1,101,1,88,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,9.0,0.0,9.0,4,2,1.0,P044,1160.0,0.0,1160.0,0.0,1160.0,0.0,1702.17
361193,3260798906,2,201,2,84,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,9.0,0.0,9.0,4,2,1.0,P044,1160.0,0.0,1160.0,0.0,1160.0,0.0,1702.17


In [72]:
indicadores_educacion.to_csv('/Dataset/indicadores_educacion.csv', index=True)
indicadores_pobreza_y_educacion.to_csv('/Dataset/indicadores_pobreza_y_educacion.csv', index=True)
concentrado.to_csv('/Dataset/concentrado.csv', index=False)
poblacion.to_csv('/Dataset/poblacion.csv', index=False)
poblacion_laboral.to_csv('/Dataset/poblacion_laboral.csv', index=False)

# Conclusiones

Ahora que el dataset se encuentra limpio, puede ser utilizado para el análisis que responderá nuestras preguntas de interés.


**Está conformado por 5 DataFrames:**
- `indicadores_educacion`
- `indicadores_pobreza_y_educacion`
- `concentrado`
- `poblacion`
- `poblacion_laboral`

Se espera que la mayor parte de la información se obtenga del DataFrame `poblacion_laboral` aunque el resto de los DataFrames se usarán para complementar la información, y validar o negar hipótesis que se puedan construir.



In [29]:
poblacion_completo.shape

(361195, 37)

Cuenta finalmente con **361,195 registros** y **37 columnas**.

Nuestra dimensión de datos es lo suficientemente grande para realizar inferencias sobre la relación entre el ingreso y educación, ya que en el dataframe final contamos con el máximo nivel de estudios de cada individuo junto con sus últimas 6 percepciones mensuales de ingresos.

Un primer paso para el análisis es crear visualizaciones. Un boxplot de los ingresos segmentado por nivel de estudios nos permite observar si existen diferencias significativas en la media de los ingresos. De esta manera se podría determinar si los ingresos de los individuos con máximo nivel de estudios de primaria, difieren con los de secundaria, por ejemplo.

Lo esperado sería que si se toma en cuenta únicamente ingresos salariales, sí se visualizaría una correlación positiva con los años de estudio. El caso interesante sería si se toma en cuenta cualquier tipo de ingreso. Este es un factor importante para los jóvenes que deciden invertir en educación formal o emprender en oficios en lugar de profesiones. 

Además del nivel de estudios, otras variables pueden tener un efecto en el ingreso percibido por los habitantes mexicanos. 
Se han realizado estudios que demuestran que el nivel socioeconómico y características raciales de un individuo influyen en el ingreso ofertado por los empleadores. Lo que demuestra la triste realidad en México (y en muchas otras partes del mundo), que la gente con un bajo nivel socioeconómico tiene deventaja en la competencia salarial.

Se pueden realizar regresiones lineales múltiples utilizando estas variables para determinar si efectivamente estos factores pueden explicar la cantidad de ingreso (salario) que perciben los individuos mexicanos.

Es intuitivo suponer que entre más años se haya preparado una persona, mejores oportunidades salariales tendría. Los mejor educados son en general más flexibles y están más motivados, se adaptan con mayor facilidad a las circunstancias que cambian, se benefician más de la experiencia profesional y del entrenamiento, actúan con mayor iniciativa en situaciones problemáticas, adquieren responsabilidades de supervisión más rápido y, a el corto plazo, son más productivos que los de menor educación, sobre todo cuando ésta no les ha proporcionando ninguna habilidad específica.

Pero es una realidad que los tiempos están cambiando y la educación ya no únicamente se ofrece en formato de licenciaturas, maestrías y doctorados. La gente se está preparando con la facilidad de distintas plataformas de cursos en línea, por ejemplo. Este tiempo invertido a la eduación aún no se contabaliza en encuestas tradicionales como las realizadas por el INEGI.

Sería de interés obtener datos sobre este tipo de educación en línea para complementar el análisis y determinar una mejor relación entre estas dos variables. 